### **Обработка данных с использованием Pickle и Parquet**

#### Задача:
Загрузка и преобразование данных с измерением производительности для двух форматов — **Pickle** и **Parquet**. Основное внимание уделяется сравнению их размера и эффективности.

#### Используемые библиотеки:
- **pandas**: Для работы с данными.
- **pyarrow**: Для работы с форматом Parquet.
- **os, psutil, time**: Для измерений (размеров файлов, использования памяти и времени).

#### Шаги выполнения:
1. **Загрузка данных**: Загружаются данные в формате Pickle.
2. **Конвертация в Parquet**: Данные преобразуются в формат Parquet для меньшего размера и лучшей производительности.
3. **Сравнение**: Измеряются размеры файлов и использование памяти для каждого формата.
4. **Результаты**: Parquet значительно экономит память и время при обработке данных по сравнению с Pickle.

#### Вывод:
Использование формата **Parquet** для больших данных дает значительную экономию в размере файлов и повышает производительность.

### Импорт

In [15]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import psutil
import time

### Функции

In [6]:
# Функции для измерения
def get_size_mb(file_path):
    """Получить размер файла в МБ"""
    return os.path.getsize(file_path) / (1024 * 1024)

def get_memory_usage():
    """Получить текущее использование памяти процессом"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Конвертация в МБ

def measure_execution_time(func):
    """Декоратор для измерения времени выполнения"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Время выполнения {func.__name__}: {end_time - start_time:.2f} секунд")
        return result
    return wrapper

# Функции для работы с данными
@measure_execution_time
def load_pickle_files(sessions_path='ga_sessions.pkl', hits_path='ga_hits.pkl'):
    """Загрузка пиклов"""
    initial_memory = get_memory_usage()

    df_sessions = pd.read_pickle(sessions_path)
    df_hits = pd.read_pickle(hits_path)

    final_memory = get_memory_usage()
    memory_used = final_memory - initial_memory

    print(f'\nРазрмеры файлов:')
    print(f'Сессии {get_size_mb(sessions_path):.2f} МБ')
    print(f'Хиты {get_size_mb(hits_path):.2f} МБ')
    print(f'Использовано памяти: {memory_used:.2f} МБ')

    return df_sessions, df_hits

@measure_execution_time
def convert_to_parquet(df, output, compression='gzip'):
    """Конвертация дф в паркет"""
    table = pa.Table.from_pandas(df)
    pq.write_table(table, output, compression=compression)
    print(f'Размер паркет файла: {get_size_mb(output):.2f} МБ')

@measure_execution_time
def load_parquet(file_path, columns=None):
    """Загрузка паркета"""
    initial_memory = get_memory_usage()

    if columns:
        df = pq.read_table(file_path, columns=columns).to_pandas()
    else:
        df = pq.read_table(file_path).to_pandas()

    final_memory = get_memory_usage()
    memory_used = final_memory - initial_memory

    print(f'Использовано памяти: {memory_used:.2f} МБ')
    return df

### Основной код выполнения

In [7]:
df_sessions_pkl, df_hits_pkl = load_pickle_files('ga_sessions.pkl', 'ga_hits.pkl')


Разрмеры файлов:
Сессии 377.11 МБ
Хиты 4191.90 МБ
Использовано памяти: 12704.75 МБ
Время выполнения load_pickle_files: 7.96 секунд


In [8]:
convert_to_parquet(df_sessions_pkl, 'ga_sessions.parquet.gz')
convert_to_parquet(df_hits_pkl, 'ga_hits.parquet.gz')

Размер паркет файла: 50.79 МБ
Время выполнения convert_to_parquet: 6.85 секунд
Размер паркет файла: 336.45 МБ
Время выполнения convert_to_parquet: 34.91 секунд


In [9]:
df_sessions_parquet = load_parquet('ga_sessions.parquet.gz')
df_hits_parquet = load_parquet('ga_hits.parquet.gz')

Использовано памяти: 2035.12 МБ
Время выполнения load_parquet: 0.98 секунд
Использовано памяти: 4231.52 МБ
Время выполнения load_parquet: 4.70 секунд


In [10]:
columns_subset = ['session_id', 'utm_medium']
df_sessions_subset = load_parquet('ga_sessions.parquet.gz', columns=columns_subset)

Использовано памяти: 597.19 МБ
Время выполнения load_parquet: 0.48 секунд


In [14]:
print(f'\nИтоговое сравнения размеров:')
print(f"Sessions pickle: {get_size_mb('ga_sessions.pkl'):.2f} МБ")
print(f"Sessions parquet: {get_size_mb('ga_sessions.parquet.gz'):.2f} МБ")
print(f"\nHits pickle: {get_size_mb('ga_hits.pkl'):.2f} МБ")
print(f"Hits parquet: {get_size_mb('ga_hits.parquet.gz'):.2f} МБ")


Итоговое сравнения размеров:
Sessions pickle: 377.11 МБ
Sessions parquet: 50.79 МБ

Hits pickle: 4191.90 МБ
Hits parquet: 336.45 МБ
